In [1]:
import pyspark
import pandas as pd
import numpy as np
# create spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *

### Create a spark data frame that contains your favorite programming languages.
- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [2]:
# create pandas dataframe

pd_df = pd.DataFrame({'language': ['python', 'julia', 'ruby', 'R', 'C++', 'Javascript', 'Typescript', 'Swift', 'Rust']})

In [3]:
# create spark dataframe from panadas dataframe
sp_df = spark.createDataFrame(pd_df)
sp_df

DataFrame[language: string]

In [4]:
# View the shema of the dataframe
sp_df.printSchema()

root
 |-- language: string (nullable = true)



In [5]:
# another way to print schema
sp_df.dtypes

[('language', 'string')]

In [6]:
#output the shape of the dataframe
sp_df.describe().show()
print("DataFrame shape: ", sp_df.count(), " x ", len(sp_df.columns))

+-------+--------+
|summary|language|
+-------+--------+
|  count|       9|
|   mean|    null|
| stddev|    null|
|    min|     C++|
|    max|    ruby|
+-------+--------+

DataFrame shape:  9  x  1


In [7]:
sp_df.show(5)

+--------+
|language|
+--------+
|  python|
|   julia|
|    ruby|
|       R|
|     C++|
+--------+
only showing top 5 rows



### Load the mpg dataset as a spark dataframe.
- Create 1 column of output that contains a message like the one below:
>- The 1999 audi a4 has a 4 cylinder engine.
>- For each vehicle.
>- Transform the trans column so that it only contains either manual or auto.

In [8]:
#Spark dataframe
import pydataset

mpg = spark.createDataFrame(pydataset.data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
mpg.select(
    mpg.year.alias("year"), col("hwy").alias("highway_mileage"),
    mpg.cty.alias("city_mileage"), mpg.cyl.alias("cylinders"))

DataFrame[year: bigint, highway_mileage: bigint, city_mileage: bigint, cylinders: bigint]

In [10]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

In [11]:
# concatenate string columns to create the desired output column
description = mpg.select(concat(lit("The "), mpg.year, (lit(" ")), mpg.manufacturer, (lit(" ")), mpg.model, (lit(" has a")), mpg.cyl, (lit(" engine"))))
description.show(20, False)

+-------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a, cyl,  engine)|
+-------------------------------------------------------------------+
|The 1999 audi a4 has a4 engine                                     |
|The 1999 audi a4 has a4 engine                                     |
|The 2008 audi a4 has a4 engine                                     |
|The 2008 audi a4 has a4 engine                                     |
|The 1999 audi a4 has a6 engine                                     |
|The 1999 audi a4 has a6 engine                                     |
|The 2008 audi a4 has a6 engine                                     |
|The 1999 audi a4 quattro has a4 engine                             |
|The 1999 audi a4 quattro has a4 engine                             |
|The 2008 audi a4 quattro has a4 engine                             |
|The 2008 audi a4 quattro has a4 engine                             |
|The 1999 audi a4 qu

In [12]:
#Transform the trans column so that it only contains either manual or auto.
mpg.select(
    regexp_extract('trans', r'^(\w+)\(', 1).alias('trans_extract'),
    regexp_replace('trans', r'\(.+$', '').alias('trans_replace'),
    when(mpg.trans.like('auto%'), 'auto').otherwise('manual').alias('trans_when')
).show()

+-------------+-------------+----------+
|trans_extract|trans_replace|trans_when|
+-------------+-------------+----------+
|         auto|         auto|      auto|
|       manual|       manual|    manual|
|       manual|       manual|    manual|
|         auto|         auto|      auto|
|         auto|         auto|      auto|
|       manual|       manual|    manual|
|         auto|         auto|      auto|
|       manual|       manual|    manual|
|         auto|         auto|      auto|
|       manual|       manual|    manual|
|         auto|         auto|      auto|
|         auto|         auto|      auto|
|       manual|       manual|    manual|
|         auto|         auto|      auto|
|       manual|       manual|    manual|
|         auto|         auto|      auto|
|         auto|         auto|      auto|
|         auto|         auto|      auto|
|         auto|         auto|      auto|
|         auto|         auto|      auto|
+-------------+-------------+----------+
only showing top

### Load the tips dataset as a spark dataframe.
- What percentage of observations are smokers?
-  Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [13]:
# Load the tips dataset

tips = spark.createDataFrame(pydataset.data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [14]:
# tips.groupBy('smoker').count().withColumn('percent',
#                                          concat(round((col('count')/tips.count()*100), 0).cast("int"), 
# lit("%"))).show()

In [15]:
# What percentage of observations are smokers
tips.groupBy('smoker').count().withColumn('percent', 
                                          concat(round((col('count')/tips.count()*100), 0).cast("int"), 
                                                 lit("%"))).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|    62%|
|   Yes|   93|    38%|
+------+-----+-------+



In [33]:
# Create a column that contains the tip percent
tip_percent = (col("tip") / col("total_bill"))*100

tips.select(
    col("total_bill").alias("total"),
    tips.tip.alias("tip_amount"),
    tip_percent.alias("tip_percent"),
).show(5)

+-----+----------+------------------+
|total|tip_amount|       tip_percent|
+-----+----------+------------------+
|16.99|      1.01|5.9446733372572105|
|10.34|      1.66|16.054158607350097|
|21.01|       3.5|16.658733936220845|
|23.68|      3.31| 13.97804054054054|
|24.59|      3.61|14.680764538430255|
+-----+----------+------------------+
only showing top 5 rows



In [36]:
# Calculate the average tip percentage for each combination of sex and smoker.
tips.groupBy('sex','smoker').count().withColumn('avg_percent', 
                                          concat(round((col('tips')/col('total_bill')*100), 0).cast("int"), 
                                                 lit("%"))).show()

AnalysisException: "cannot resolve '`tips`' given input columns: [sex, smoker, count];;\n'Project [sex#168, smoker#169, count#440L, concat(cast(round((('tips / 'total_bill) * 100), 0) as int), %) AS avg_percent#444]\n+- Aggregate [sex#168, smoker#169], [sex#168, smoker#169, count(1) AS count#440L]\n   +- LogicalRDD [total_bill#166, tip#167, sex#168, smoker#169, day#170, time#171, size#172L], false\n"

In [29]:
avg_tip = ((col("tip") / col("total_bill"))*100)


In [32]:
tips.select(
    col("total_bill").alias("total"),
    tips.tip.alias("tip_amount"),
    tip_percent.alias("tip_percent"),


SyntaxError: invalid syntax (<ipython-input-32-e2863523ab6e>, line 5)

### Use the seattle weather dataset referenced in the lesson to answer the questions below.
- Convert the temperatures to farenheight.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low tempurature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation)